In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install pandas

In [19]:
import requests
from bs4 import BeautifulSoup
import os
import json
import pandas as pd
from urllib.parse import urljoin

BASE_URL = 'https://store.steampowered.com'

def create_directory(directory):
    try:
        os.mkdir(directory)
    except FileExistsError:
        pass

def get_total_items(soup):
    try:
        page_item = soup.find('div', 'search_pagination_right').find_all('a')
        return int(page_item[0].text)
    except Exception as e:
        print(f"Error getting total items: {e}")
        return 0

def get_pagination(term, page):
    params = {
        'term': term,
        'page': page,
    }

    req = requests.get(urljoin(BASE_URL, '/search/results'), params=params, headers={'cookie': 'sessionid=cd46137aee87759ca68f1347'})
    soup = BeautifulSoup(req.text, 'html.parser')

    total_items = get_total_items(soup)
    return total_items

def scrape_game_info(game_url):
    game_req = requests.get(urljoin(BASE_URL, game_url))
    game_soup = BeautifulSoup(game_req.text, 'html.parser')

    game_title_elem = game_soup.find('div', 'apphub_AppName')
    game_title = game_title_elem.text.strip() if game_title_elem else 'none'

    game_description_elem = game_soup.find('div', {'id': 'game_area_description'})
    game_description = game_description_elem.text.strip() if game_description_elem else 'none'

    return {
        'title': game_title,
        'description': game_description,
    }

def scrape_similar_games(similar_games_url):
    similar_games_req = requests.get(urljoin(BASE_URL, similar_games_url))
    similar_games_soup = BeautifulSoup(similar_games_req.text, 'html.parser')

    similar_games = []

    similar_games_content = similar_games_soup.find_all('a', {'class': 'search_result_row'})
    for similar_game in similar_games_content:
        similar_game_title = similar_game.find('span', {'class': 'title'}).text.strip()
        similar_game_link = similar_game['href']

        similar_game_data = {
            'title': similar_game_title,
            'link': similar_game_link,
        }
        similar_games.append(similar_game_data)

    return similar_games

def scrap(term):
    create_directory('resultfile')
    count = 0
    datas = []

    for j in range(1, get_pagination(term, 1) + 1):
        params = {
            'term': term,
            'page': j,
        }
        req = requests.get(urljoin(BASE_URL, '/search/results'), params=params, headers={'cookie': 'sessionid=cd46137aee87759ca68f1347'})
        soup = BeautifulSoup(req.text, 'html.parser')

        content = soup.find('div', {'id': 'search_resultsRows'}).find_all('a')
        for i in content:
            url = i['href']
            title = i.find('div', 'col search_name ellipsis').text.strip().replace('\n', ' ')

            try:
                price_elem = i.find('div', 'col search_price responsive_secondrow')
                price = price_elem.text.strip() if price_elem else 'none'
            except Exception:
                price_elem = i.find('div', 'col search_price discounted responsive_secondrow')
                if price_elem:
                    price = (
                        'discount from ' +
                        i.find('span', {'style': 'color: #888888;'}).text +
                        ' to ' +
                        price_elem.find('br').next_sibling.strip() +
                        f" ({i.find('div', 'col search_discount responsive_secondrow').text.replace('-', '').strip()})"
                    )
                else:
                    price = 'none'

            release = i.find('div', 'col search_released responsive_secondrow').text if i.find('div', 'col search_released responsive_secondrow') else 'none'

            data = {
                'title': title,
                'price': price,
                'release': release,
                'link': url
            }
            datas.append(data)

            # Scrape additional game information
            game_info = scrape_game_info(url)
            data.update(game_info)

            # Scrape similar games
            similar_games_url_elem = i.find('a', {'class': 'search_capsule'})
            if similar_games_url_elem:
                similar_games_url = urljoin(BASE_URL, similar_games_url_elem['href'] + '/recommendations')
                similar_games = scrape_similar_games(similar_games_url)
                data['similar_games'] = similar_games
            else:
                data['similar_games'] = []

            count += 1
            print(f'{count}. {title}. released: {release}. price: {price} . link: {url}')

    with open(f'resultfile/json_data_{term}.json', 'w+') as outfile:
        json.dump(datas, outfile)

    df = pd.DataFrame(datas)
    df.to_csv(f'resultfile/csv_data_{term}.csv', index=False)
    df.to_excel(f'resultfile/excel_data_{term}.xlsx', index=False)
    print('All data was created')

def run():
    term = input('Enter the game you want to search: ')
    scrap(term)

run()


Enter the game you want to search: VALHEIM
1. Valheim. released: 
                    Feb 2, 2021                . price: none . link: https://store.steampowered.com/app/892970/Valheim/?snr=1_7_7_151_150_1
2. Valheim Soundtrack. released: 
                    Oct 29, 2021                . price: none . link: https://store.steampowered.com/app/1620250/Valheim_Soundtrack/?snr=1_7_7_151_150_1
3. Valheim Soundtrack Bundle. released: 
. price: none . link: https://store.steampowered.com/bundle/24016/Valheim_Soundtrack_Bundle/?snr=1_7_7_151_150_1
4. Sunkenland. released: 
                    Aug 25, 2023                . price: none . link: https://store.steampowered.com/app/2080690/Sunkenland/?snr=1_7_7_151_150_1
5. Streets of Rogue 2. released: 
                    2024                . price: none . link: https://store.steampowered.com/app/2165810/Streets_of_Rogue_2/?snr=1_7_7_151_150_1
6. Pathless Woods. released: 
                    2024                . price: none . link: https://sto